In [1]:

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
warnings.filterwarnings('ignore')
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import glob
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Conv2D  ,MaxPooling2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
import tensorflow as tf


### Preparing Data

In [16]:
respeck_filepaths = glob.glob("../Respeck/*")
df1 = pd.DataFrame()
for rfp in respeck_filepaths:
    files = glob.glob(f"{rfp}/*")
    
    for file in files:
        # [main_act,sub_act] = file.split(".csv")[0].split('_')[-2:]
        main_activity = " ".join(file.split(".csv")[0].split('_')[-2:])
        
        df = pd.read_csv(file,index_col=0)
        df['activity'] = main_activity
        df['user'] = rfp.split('\\')[-1]
        # print(df)
        df1 = df1.append(df)


In [17]:
df1['activity'] = df1['activity'].apply(lambda x: x.replace('standing','sitting/standing'))
df1['activity'] = df1['activity'].apply(lambda x: x.replace('sitting ','sitting/standing '))
df1

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,activity,user
0,1697605965,0.011963,-0.855774,-0.029846,-9.625000,-0.859375,-5.656250,ascending breathingNormal,s1
1,1697606005,-0.001709,-0.826233,-0.036194,0.359375,4.953125,-1.656250,ascending breathingNormal,s1
2,1697606045,-0.058838,-0.933899,-0.032532,3.437500,9.406250,-2.031250,ascending breathingNormal,s1
3,1697606085,-0.002441,-1.115051,-0.028870,3.078125,6.921875,-5.984375,ascending breathingNormal,s1
4,1697606125,-0.036621,-1.035217,-0.076477,4.328125,8.125000,0.625000,ascending breathingNormal,s1
...,...,...,...,...,...,...,...,...,...
755,1697636165,-0.206299,-0.975891,0.181335,0.859375,-1.125000,-0.625000,sitting/standing talking,s98
756,1697636205,-0.196289,-0.944153,0.163513,0.812500,0.390625,0.203125,sitting/standing talking,s98
757,1697636245,-0.194824,-0.959290,0.168640,1.390625,0.421875,0.312500,sitting/standing talking,s98
758,1697636285,-0.191406,-0.976868,0.154480,0.546875,0.859375,-0.406250,sitting/standing talking,s98


In [18]:
columns = ['user','activity','timestamp', 'accel_x', 'accel_y', 'accel_z']
# df1 = df1[columns]
df_har = df1[columns]
# removing null values
df_har = df_har.dropna()
df_har.shape
# transforming the user to float
df_har['user'] = df_har['user'].str.replace('s', '')
df_har['user'] = df_har['user'].apply(lambda x:int(x))


In [13]:
classes = ['lyingBack breathingNormal', 'lyingBack coughing',
       'lyingBack hyperventilating', 'lyingBack laughing',
       'lyingBack singing', 'lyingBack talking',
       'lyingLeft breathingNormal', 'lyingLeft coughing',
       'lyingLeft hyperventilating', 'lyingLeft laughing',
       'lyingLeft singing', 'lyingLeft talking',
       'lyingRight breathingNormal', 'lyingRight coughing',
       'lyingRight hyperventilating', 'lyingRight laughing',
       'lyingRight singing', 'lyingRight talking',
       'lyingStomach breathingNormal', 'lyingStomach coughing',
       'lyingStomach hyperventilating', 'lyingStomach laughing',
       'lyingStomach singing', 'lyingStomach talking',
       'sitting/standing breathingNormal', 'sitting/standing coughing',
       'sitting/standing eating', 'sitting/standing hyperventilating',
       'sitting/standing laughing', 'sitting/standing singing',
       'sitting/standing talking']


df_har = df_har[df_har['activity'].isin(classes)] 

In [26]:
df_har.to_csv('task3.csv',index=False)

In [2]:
# ONLY RUN THIS AFTER CSV GENERATION
all_df = pd.read_csv('task3.csv')
all_df

,user,activity,timestamp,accel_x,accel_y,accel_z
0,1,lyingBack breathingNormal,1697605965,-0.596436,-0.181213,0.870056
1,1,lyingBack breathingNormal,1697606005,-0.593994,-0.189026,0.880310
2,1,lyingBack breathingNormal,1697606045,-0.591064,-0.172668,0.883484
3,1,lyingBack breathingNormal,1697606085,-0.580322,-0.176819,0.875671
4,1,lyingBack breathingNormal,1697606125,-0.598145,-0.189758,0.882263
...,...,...,...,...,...,...
2031239,98,sitting/standing talking,1697636165,-0.206299,-0.975891,0.181335
2031240,98,sitting/standing talking,1697636205,-0.196289,-0.944153,0.163513
2031241,98,sitting/standing talking,1697636245,-0.194824,-0.959290,0.168640
2031242,98,sitting/standing talking,1697636285,-0.191406,-0.976868,0.154480


In [5]:
all_df['activity'].unique()


array(['lyingBack breathingNormal', 'lyingBack coughing',
       'lyingBack hyperventilating', 'lyingBack laughing',
       'lyingBack singing', 'lyingBack talking',
       'lyingLeft breathingNormal', 'lyingLeft coughing',
       'lyingLeft hyperventilating', 'lyingLeft laughing',
       'lyingLeft singing', 'lyingLeft talking',
       'lyingRight breathingNormal', 'lyingRight coughing',
       'lyingRight hyperventilating', 'lyingRight laughing',
       'lyingRight singing', 'lyingRight talking',
       'lyingStomach breathingNormal', 'lyingStomach coughing',
       'lyingStomach hyperventilating', 'lyingStomach laughing',
       'lyingStomach singing', 'lyingStomach talking',
       'sitting/standing breathingNormal', 'sitting/standing coughing',
       'sitting/standing eating', 'sitting/standing hyperventilating',
       'sitting/standing laughing', 'sitting/standing singing',
       'sitting/standing talking'], dtype=object)

### Segment Generation

In [3]:
random_seed = 42   
n_time_steps = 150 
n_features = 3 
step = 10
n_epochs = 5      
batch_size = 32

In [4]:
class PDIOTLabelEncoder:
    def __init__(self):
        self.classes = ['lyingBack breathingNormal', 'lyingBack coughing',
       'lyingBack hyperventilating', 'lyingBack laughing',
       'lyingBack singing', 'lyingBack talking',
       'lyingLeft breathingNormal', 'lyingLeft coughing',
       'lyingLeft hyperventilating', 'lyingLeft laughing',
       'lyingLeft singing', 'lyingLeft talking',
       'lyingRight breathingNormal', 'lyingRight coughing',
       'lyingRight hyperventilating', 'lyingRight laughing',
       'lyingRight singing', 'lyingRight talking',
       'lyingStomach breathingNormal', 'lyingStomach coughing',
       'lyingStomach hyperventilating', 'lyingStomach laughing',
       'lyingStomach singing', 'lyingStomach talking',
       'sitting/standing breathingNormal', 'sitting/standing coughing',
       'sitting/standing eating', 'sitting/standing hyperventilating',
       'sitting/standing laughing', 'sitting/standing singing',
       'sitting/standing talking']
        
        self.mappings = {c:i for i,c in enumerate(self.classes)}
    
    def fit_transform(self,y):
        return np.array([self.mappings[c] for c in y])
    
    def inverse_transform(self, y):
        reverse_mapping = {v: k for k, v in self.mappings.items()}
        return [reverse_mapping[val] for val in y]
        
        

In [5]:
def segments_overlap(data,n_time_steps=n_time_steps,step=step):
    segments = []
    labels = []

    for i in range(0,  all_df.shape[0]- n_time_steps, step):  

        xs = all_df['accel_x'].values[i: i + n_time_steps]

        ys = all_df['accel_y'].values[i: i + n_time_steps]

        zs = all_df['accel_z'].values[i: i + n_time_steps]

        label = stats.mode(all_df['activity'][i: i + n_time_steps])[0][0]

        segments.append([xs, ys, zs])

        labels.append(label)
        
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)
    labels = np.asarray(labels)
    # print(labels)
    le = PDIOTLabelEncoder()
    labels = le.fit_transform(labels)
    
    # enc = OneHotEncoder(handle_unknown='ignore').fit(labels)
    # labels = enc.transform(labels).toarray()
    # labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    # print(enc.categories_)
    return reshaped_segments,labels,le.classes

In [180]:
def segments_no_overlap(data):
    segments= []
    labels = []
    
    activities = data['activity'].unique()
    for act in activities:
        act_data = data[data['activity'] == act]
        
        # for i in range(0, len(all_data) - n_time_steps, step):
        for i in range(0,  act_data.shape[0]- n_time_steps, step):  

            xs = act_data['accel_x'].values[i: i + n_time_steps]
            ys = act_data['accel_y'].values[i: i + n_time_steps]
            zs = act_data['accel_z'].values[i: i + n_time_steps]

            segments.append([xs, ys, zs])
            labels.append(act)

    #reshape the segments which is (list of arrays) to a list
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)
    labels_dummies = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    
    return reshaped_segments,labels

### Model Training

In [182]:
def model_LSTM(X_train,y_train):
    model = Sequential()
    # RNN layer
    model.add(LSTM(units = 128, input_shape = (X_train.shape[1], X_train.shape[2])))
    # Dropout layer
    model.add(Dropout(0.5)) 
    # Dense layer with ReLu
    model.add(Dense(units = 64, activation='relu'))
    # Softmax layer
    model.add(Dense(y_train.shape[1], activation = 'softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model 

In [183]:
def model_cnn(trainX, trainy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=n_epochs, batch_size=batch_size, verbose=1)
    # evaluate model
    return model

In [184]:
def model_cnn2(trainX, trainy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_time_steps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=20, batch_size=128, verbose=1)
    # evaluate model
    return model

In [185]:
def model_cnn3(trainX,trainy):
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation = 'relu', input_shape = (n_time_steps,n_features)))
    model.add(Dropout(0.1))

    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(44, activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.fit(trainX, trainy, epochs =10, batch_size=32, verbose=1)
    return model

In [16]:
def model_cnn4(X_train, y_train,n_time_steps=n_time_steps):
    m = Sequential()
    m.add(Conv1D(64, 3, activation='relu', input_shape=(n_time_steps, 3)))
    m.add(Conv1D(64, 3, activation='relu'))
    m.add(MaxPooling1D(3))
    m.add(Conv1D(128, 3, activation='relu'))
    m.add(Conv1D(128, 3, activation='relu'))
    m.add(MaxPooling1D(3))
    m.add(Dropout(0.5))
    m.add(Flatten())
    m.add(Dense(100, activation='relu'))
    m.add(Dropout(0.5))
    m.add(Dense(31, activation='softmax')) # Change this to the number of classes you have

    # Compile model
    m.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    m.fit(X_train, y_train, epochs = 20, validation_split=0.2, verbose=1)
    return m

### Driver Code

In [21]:
accuracies = []
for user in all_df['user'].unique():
    all_df = all_df
    train_df = all_df[all_df['user'] != user]
    test_df = all_df[all_df['user'] == user]

    X_train, y_train, categories = segments_overlap(train_df)
    X_test, y_test, _ = segments_overlap(test_df)
    
    model = model_cnn4(X_train,y_train)
    
    loss, accuracy = model.evaluate(X_test, y_test, verbose = 1)
    print(f"Test Accuracy ({user}):", accuracy)
    print(f"Test Loss ({user}):", loss)

    accuracies.append((user,loss,accuracy))
    
    break
    

Epoch 1/20
5078/5078 [==============================] - 90s 18ms/step - loss: 1.5366 - accuracy: 0.3496 - val_loss: 1.5007 - val_accuracy: 0.4617
Epoch 2/20
5078/5078 [==============================] - 86s 17ms/step - loss: 1.2024 - accuracy: 0.4860 - val_loss: 1.4457 - val_accuracy: 0.5253
Epoch 3/20
5078/5078 [==============================] - 91s 18ms/step - loss: 1.0606 - accuracy: 0.5591 - val_loss: 1.4128 - val_accuracy: 0.5716
Epoch 4/20
5078/5078 [==============================] - 93s 18ms/step - loss: 0.9723 - accuracy: 0.5977 - val_loss: 1.4303 - val_accuracy: 0.5950
Epoch 5/20
5078/5078 [==============================] - 87s 17ms/step - loss: 0.9165 - accuracy: 0.6208 - val_loss: 1.4387 - val_accuracy: 0.5993
Epoch 6/20
5078/5078 [==============================] - 87s 17ms/step - loss: 0.8665 - accuracy: 0.6414 - val_loss: 1.4294 - val_accuracy: 0.6046
Epoch 7/20
5078/5078 [==============================] - 86s 17ms/step - loss: 0.8321 - accuracy: 0.6541 - val_loss: 1.4890 -

In [17]:
def testing():
    accuracies = []
    n_epochs = 5 
    tesing_all_df = pd.read_csv('task3.csv')
    for s in [10]:
        for w in [125]:
            n_time_steps = w  
            step = s
            # print(all_df)
            for user in tesing_all_df['user'].unique():

                train_df = tesing_all_df[tesing_all_df['user'] != user]
                test_df = tesing_all_df[tesing_all_df['user'] == user]

                X_train, y_train, categories = segments_overlap(train_df,w,s)
                X_test, y_test, _ = segments_overlap(test_df,w,s)
                print(X_train.shape)
                
                model = model_cnn4(X_train,y_train,w)
                
                loss, accuracy = model.evaluate(X_test, y_test, verbose = 1)
                print(f"Test Accuracy ({user}):", accuracy)
                print(f"Test Loss ({user}):", loss)

                accuracies.append((s,w,loss,accuracy))
                print(w,s,accuracy)
                
                return model 
            
m = testing()

(203112, 125, 3)
Epoch 1/20
5078/5078 [==============================] - 104s 20ms/step - loss: 1.4921 - accuracy: 0.3727 - val_loss: 1.5488 - val_accuracy: 0.4586
Epoch 2/20
5078/5078 [==============================] - 108s 21ms/step - loss: 1.1796 - accuracy: 0.5070 - val_loss: 1.3879 - val_accuracy: 0.5486
Epoch 3/20
5078/5078 [==============================] - 101s 20ms/step - loss: 1.0540 - accuracy: 0.5641 - val_loss: 1.3470 - val_accuracy: 0.5739
Epoch 4/20
5078/5078 [==============================] - 99s 20ms/step - loss: 0.9737 - accuracy: 0.5987 - val_loss: 1.3924 - val_accuracy: 0.6039
Epoch 5/20
5078/5078 [==============================] - 99s 20ms/step - loss: 0.9250 - accuracy: 0.6183 - val_loss: 1.4265 - val_accuracy: 0.6086
Epoch 6/20
5078/5078 [==============================] - 99s 20ms/step - loss: 0.8833 - accuracy: 0.6343 - val_loss: 1.4547 - val_accuracy: 0.6089
Epoch 7/20
5078/5078 [==============================] - 101s 20ms/step - loss: 0.8512 - accuracy: 0.6468

In [15]:

print(accuracies)

NameError: name 'accuracies' is not defined

In [23]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(f'../models/cnn_model_task{3}_{n_time_steps}_{step}.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmpes4vtp6m\assets


INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmpes4vtp6m\assets
